In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request as request
import csv

In [2]:
#Importing educational funding data from the EuroStat database
edu = pd.read_csv('educ_figdp_1_Data.csv', na_values = ':',usecols = ["TIME","GEO","Value"])

In [3]:
edu.head()

,TIME,GEO,Value
0,2000,European Union (28 countries),NaN
1,2001,European Union (28 countries),NaN
2,2002,European Union (28 countries),5.00
3,2003,European Union (28 countries),5.03
4,2004,European Union (28 countries),4.95


In [4]:
edu.describe()

,TIME,Value
count,384.000000,361.000000
mean,2005.500000,5.203989
std,3.456556,1.021694
min,2000.000000,2.880000
25%,2002.750000,4.620000
50%,2005.500000,5.060000
75%,2008.250000,5.660000
max,2011.000000,8.810000


In [5]:
edu[90:94]

,TIME,GEO,Value
90,2006,Belgium,5.98
91,2007,Belgium,6.00
92,2008,Belgium,6.43
93,2009,Belgium,6.57


In [6]:
#loc for label-based indexing
edu.loc[90:94, ['TIME','GEO']]

,TIME,GEO
90,2006,Belgium
91,2007,Belgium
92,2008,Belgium
93,2009,Belgium
94,2010,Belgium


In [7]:
#filtering data via Boolean indexing
edu[edu['Value'] > 6.5].tail()

,TIME,GEO,Value
286,2010,Malta,6.74
287,2011,Malta,7.96
381,2009,Finland,6.81
382,2010,Finland,6.85
383,2011,Finland,6.76


In [8]:
#to find null values
edu[edu["Value"].isnull()].head()

,TIME,GEO,Value
0,2000,European Union (28 countries),NaN
1,2001,European Union (28 countries),NaN
36,2000,Euro area (18 countries),NaN
37,2001,Euro area (18 countries),NaN
48,2000,Euro area (17 countries),NaN


<b> Manupulating Data </b>

In [9]:
#Lambda fn (inline fn) that squares all the entries in the value series;  the lambda
#fn will return the square of all the values
s = edu["Value"].apply(lambda d: d**2)
s.head()

0        NaN
1        NaN
2    25.0000
3    25.3009
4    24.5025
Name: Value, dtype: float64

<b> Setting new values into the DataFrame </b>

In [10]:
#In the following example, we assign the Series that results from dividing the 
#Value column by the maximum value in the same column to a new column named ValueNorm.
edu['ValueNorm'] = edu['Value']/edu['Value'].max() 
edu.tail()

,TIME,GEO,Value,ValueNorm
379,2007,Finland,5.90,0.669694
380,2008,Finland,6.10,0.692395
381,2009,Finland,6.81,0.772985
382,2010,Finland,6.85,0.777526
383,2011,Finland,6.76,0.767310


In [11]:
edu.drop('ValueNorm', axis = 1, inplace = True) 
edu.head()

,TIME,GEO,Value
0,2000,European Union (28 countries),NaN
1,2001,European Union (28 countries),NaN
2,2002,European Union (28 countries),5.00
3,2003,European Union (28 countries),5.03
4,2004,European Union (28 countries),4.95


In [12]:
#Inserting a new row into the DataFrame
#The fn receives represented as a dictionary where the keys are 
#the name of the columns and the values are the associated value
edu = edu.append({"TIME": 2000,"Value": 5.00,"GEO": 'a'}, ignore_index = True)
edu.tail()

,TIME,GEO,Value
380,2008,Finland,6.10
381,2009,Finland,6.81
382,2010,Finland,6.85
383,2011,Finland,6.76
384,2000,a,5.00


In [13]:
# Removing the last row by axis = 0
# and setting the max function over the indexes
edu.drop(max(edu.index), axis = 0, inplace = True) 
edu.tail()

,TIME,GEO,Value
379,2007,Finland,5.90
380,2008,Finland,6.10
381,2009,Finland,6.81
382,2010,Finland,6.85
383,2011,Finland,6.76


<b> Removing NaN values </b>

In [14]:
eduDrop = edu.dropna(how = 'any', subset = ["Value"]) 
eduDrop.head()

,TIME,GEO,Value
2,2002,European Union (28 countries),5.00
3,2003,European Union (28 countries),5.03
4,2004,European Union (28 countries),4.95
5,2005,European Union (28 countries),4.92
6,2006,European Union (28 countries),4.91


<b> Filling NaN values </b>

In [15]:
#For filling specific columns: 
# we have to set as argument to the fillna() function a dictionary with the name of the c
#olumns as the key and which character to 
#be used for filling as the value.
eduFilled = edu.fillna(value = {"Value": 0})
eduFilled.head()

,TIME,GEO,Value
0,2000,European Union (28 countries),0.00
1,2001,European Union (28 countries),0.00
2,2002,European Union (28 countries),5.00
3,2003,European Union (28 countries),5.03
4,2004,European Union (28 countries),4.95


<b> Sorting  </b>

In [18]:
#Sorting by columns
#data sorted from largest to smallest (descending)
edu.sort_values(by = 'Value', ascending = False, inplace = True)
edu.head()

,TIME,GEO,Value
130,2010,Denmark,8.81
131,2011,Denmark,8.75
129,2009,Denmark,8.74
121,2001,Denmark,8.44
122,2002,Denmark,8.44


In [19]:
# Data sorted in ascending order
edu.sort_index(axis = 0, ascending = True, inplace = True) 
edu.head()

,TIME,GEO,Value
0,2000,European Union (28 countries),NaN
1,2001,European Union (28 countries),NaN
2,2002,European Union (28 countries),5.00
3,2003,European Union (28 countries),5.03
4,2004,European Union (28 countries),4.95


<b> Grouping Data </b>

In [21]:
#Grouping data by country
#Use Case:
#For example, in our case, if we want a DataFrame 
#showing the mean of the values for each country over all the years, 
#we can obtain it by grouping according to country and using the 
#mean function as the aggregation method for each group
group = edu[["GEO", "Value"]].groupby('GEO').mean() 
group.head()

,Value
GEO,
Austria,5.618333
Belgium,6.189091
Bulgaria,4.093333
Cyprus,7.023333
Czech Republic,4.168333


<b> Rearranging Data </b>
<b> PIVOT TABLE! </b>

In [33]:
filtered_data = edu[edu["TIME"] > 2005]
pivedu = pd.pivot_table(filtered_data, values = 'Value',
                        index = ['GEO'], columns = ['TIME'])
pivedu

TIME,2006,2007,2008,2009,2010,2011
GEO,,,,,,
Austria,5.40,5.33,5.47,5.98,5.91,5.80
Belgium,5.98,6.00,6.43,6.57,6.58,6.55
Bulgaria,4.04,3.88,4.44,4.58,4.10,3.82
Cyprus,7.02,6.95,7.45,7.98,7.92,7.87
Czech Republic,4.42,4.05,3.92,4.36,4.25,4.51
Denmark,7.97,7.81,7.68,8.74,8.81,8.75
Estonia,4.70,4.72,5.61,6.03,5.66,5.16
Euro area (13 countries),4.87,4.80,4.94,5.32,5.28,5.15
Euro area (15 countries),4.87,4.81,4.95,5.32,5.29,5.16


In [30]:
#Now we can use the new index to select specific rows by label, using the loc operator:
pivedu.loc[['Spain','Portugal'], [2006,2011]]

TIME,2006,2011
GEO,,
Spain,4.26,4.82
Portugal,5.07,5.27


In [37]:
# Ranking Data
# Ex: We want to know how each country is ranked by year
# parameter ascending=False makes 
#the ranking go from the highest values to the lowest values.



pivedu = pivedu.drop([
'Euro area (13 countries)', 'Euro area (15 countries)', 'Euro area (17 countries)', 
    'Euro area (18 countries)', 'European Union (25 countries)',
    'European Union (27 countries)', 'European Union (28 countries)'],axis = 0)
pivedu = pivedu.rename(index = {'Germany (until 1990 former territory of the FRG)': 'Germany'})
pivedu = pivedu.dropna()
pivedu.rank(ascending = False, method = 'first').head()

TIME,2006,2007,2008,2009,2010,2011
GEO,,,,,,
Austria,10.0,7.0,11.0,7.0,8.0,8.0
Belgium,5.0,4.0,3.0,4.0,5.0,5.0
Bulgaria,21.0,21.0,20.0,20.0,22.0,22.0
Cyprus,2.0,2.0,2.0,2.0,2.0,3.0
Czech Republic,19.0,20.0,21.0,21.0,20.0,19.0
